In [5]:
# import pandas
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.lda import LDA
from sklearn.qda import QDA
import tensorflow as tf

import pandas 
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib
matplotlib.style.use('ggplot')

#Chaikin Oscillator  
def Chaikin(df):  
    ad = (2 * df['ClosePrice'] - df['HighPrice'] - df['LowPrice']) / (df['HighPrice'] - df['LowPrice']) * df['TotalVolume']  
    Chaikin = pd.Series(pd.ewma(ad, span = 3, min_periods = 2) - pd.ewma(ad, span = 10, min_periods = 9), name = 'Chaikin')  
    df = df.join(Chaikin)  
    return df

## Bolinger Bands
def bbands(price, length=10, numsd=2):
    """ returns average, upper band, and lower band"""
    ave = pd.stats.moments.rolling_mean(price,length)
    sd = pd.stats.moments.rolling_std(price,length)
    upband = ave + (sd*numsd)
    dnband = ave - (sd*numsd)
    return np.round(ave,3), np.round(upband,3), np.round(dnband,3)


#Create output labels
def label(df, numberOfRowsAhead, priceDiffRange):
    # Create a list to store the data
    labelPosChang = []
    labelNegChang = []
    labelNoChang = []
    for i in range(0, len(df)):
       price=df.iloc[i]['ClosePrice']
       rowAheadCount=0
       if i == len(df)-1:
         print("End  of array")  
         labelPosChang.append(0)
         labelNegChang.append(0)
         labelNoChang.append(1)
         break 
          
       for j in range(i+1,i+numberOfRowsAhead):
        if j > len(df)-1:
           print("End  of array")  
           labelPosChang.append(0)
           labelNegChang.append(0)
           labelNoChang.append(1)
           break 
        priceDiff=df.iloc[j]['ClosePrice'] - price
         
        rowAheadCount = rowAheadCount +1 
        if priceDiff > priceDiffRange:
           labelPosChang.append(1)
           labelNegChang.append(0)
           labelNoChang.append(0)
           break
           
        elif priceDiff < -1*priceDiffRange:
           labelPosChang.append(0)
           labelNegChang.append(1)
           labelNoChang.append(0)
           break 
        elif numberOfRowsAhead-1 == rowAheadCount:
           labelPosChang.append(0)
           labelNegChang.append(0)
           labelNoChang.append(1)
           break
          
    
    df['Poschg'] = labelPosChang
    df['Negchg'] = labelNegChang
    df['Nochg'] = labelNoChang
   
    return df





data = pandas.read_csv('C://Users/thame_000/DailyTickSandPNov112017.csv')
data=data[data['Ticker']=='MSFT']
data=Chaikin(data)
data['priceChange'] = data['ClosePrice']- data['ClosePrice'].shift() 
data['priceChangePred'] = data['ClosePrice'].shift(-1) - data['ClosePrice']
 
data=label(data,5,0.02)
 
          
 
data['movingAverage'], data['upperBB'], data['lowerBB'] = bbands(data.ClosePrice, length=30, numsd=1)
data['movingAverage5'], data['upperBB5'], data['lowerBB5'] = bbands(data.ClosePrice, length=5, numsd=1)

data.fillna(0, inplace=True)
data.drop(data.head(15).index, inplace=True)
#data.to_csv('C://Users/thame_000/OutDataNew15.csv')
columnList=['OpenPrice','HighPrice','LowPrice','TotalVolume','priceChange','movingAverage','upperBB','lowerBB','movingAverage5','upperBB5','lowerBB5','Chaikin']
#pd.scatter_matrix(np.log(data[columnListAll]), alpha = 0.3, figsize = (14,8), diagonal = 'kde');

## Outlier detection and removal
from scipy import stats
outlier=data[(np.abs(stats.zscore(data[columnList])) > 3).all(axis=1)]
data=data[(np.abs(stats.zscore(data[columnList])) < 3).all(axis=1)]
#plt.figure(); data[columnList].plot();
 
y=data[['Poschg','Negchg','Nochg']]
X=data[columnList]


X.describe()

X.corr()

C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:20: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(ignore_na=False,span=3,adjust=True,min_periods=2).mean()
C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:20: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(ignore_na=False,span=10,adjust=True,min_periods=9).mean()


End  of array
End  of array
End  of array
End  of array


C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:27: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=30).mean()
C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:28: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=30).std()
C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:27: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=5).mean()
C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:28: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=5).std()


,OpenPrice,HighPrice,LowPrice,TotalVolume,priceChange,movingAverage,upperBB,lowerBB,movingAverage5,upperBB5,lowerBB5,Chaikin
OpenPrice,1.000000,0.991073,0.992109,-0.354858,-0.077751,0.881564,0.871194,0.863522,0.987870,0.982638,0.984945,-0.080823
HighPrice,0.991073,1.000000,0.992661,-0.335710,0.024640,0.874924,0.868164,0.853638,0.983439,0.980587,0.978256,-0.030580
LowPrice,0.992109,0.992661,1.000000,-0.391638,0.020196,0.881548,0.869467,0.865125,0.984828,0.977880,0.983581,-0.027562
TotalVolume,-0.354858,-0.335710,-0.391638,1.000000,-0.067439,-0.349466,-0.288162,-0.396973,-0.365440,-0.334954,-0.391867,-0.084983
priceChange,-0.077751,0.024640,0.020196,-0.067439,1.000000,-0.031126,-0.027809,-0.033148,-0.042174,-0.041648,-0.042340,0.664222
movingAverage,0.881564,0.874924,0.881548,-0.349466,-0.031126,1.000000,0.983117,0.984489,0.905205,0.896377,0.906411,-0.008353
upperBB,0.871194,0.868164,0.869467,-0.288162,-0.027809,0.983117,1.000000,0.935777,0.895236,0.892003,0.891132,-0.010032
lowerBB,0.863522,0.853638,0.865125,-0.396973,-0.033148,0.984489,0.935777,1.000000,0.886047,0.872163,0.892279,-0.006390
movingAverage5,0.987870,0.983439,0.984828,-0.365440,-0.042174,0.905205,0.895236,0.886047,1.000000,0.995737,0.996043,-0.093875
upperBB5,0.982638,0.980587,0.977880,-0.334954,-0.041648,0.896377,0.892003,0.872163,0.995737,1.000000,0.983600,-0.101964


In [ ]:
## work on CNN